# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
from utils.ml import *
import winsound

# Action Points

### 1. No need to make 10 transactions a day. Just one is enough
### 2. Close diff (Movement of close value over a day) looks promising to find trend or ranging market

data                = get_date_list(data)
data                = read_data(data)

In [2]:
%%time
data = run_engine(data)

100%|██████████████████████████████████| 74387/74387 [00:18<00:00, 3998.76it/s]


pl    0.12863
dtype: float64
--------------------------------------
            DateTime_frmt  order_side  close_type       pl
65420 2020-11-18 04:10:00         NaN  sema_close  0.00513
65943 2020-11-19 23:45:00         NaN  sema_close -0.00328
66181 2020-11-20 19:35:00         NaN  sema_close -0.00187
66500 2020-11-23 22:10:00         NaN  sema_close -0.00595
67302 2020-11-26 17:00:00         NaN  sema_close  0.00227
67951 2020-11-30 23:05:00         NaN  sema_close -0.00077
69106 2020-12-06 23:20:00         NaN  sema_close  0.00821
69814 2020-12-09 10:20:00         NaN  sema_close -0.00110
70472 2020-12-11 17:10:00         NaN  sema_close -0.00162
71905 2020-12-18 16:35:00         NaN  sema_close  0.00657
72281 2020-12-21 23:55:00         NaN  sema_close -0.00069
72482 2020-12-22 16:40:00         NaN  sema_close -0.00781
72758 2020-12-23 15:40:00         NaN  sema_close -0.00350
73087 2020-12-24 19:05:00         NaN  sema_close -0.00286
74305 2020-12-31 15:10:00         NaN  sema_clo

100%|██████████████████████████████████████| 223/223 [00:00<00:00, 1242.95it/s]

Wall time: 1min 45s


In [3]:
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')
monthly_avg = np.round(data['df_ohlc']['pl'].sum()/len(set(data['df_ohlc']['month_val'])),4)
print(f'monthly_avg : {monthly_avg}')

month_val
1     0.00381
2     0.01905
3     0.07279
4    -0.00276
5     0.01922
6     0.01887
7     0.00787
8    -0.01320
9     0.01088
10   -0.01633
11    0.00694
12    0.00149
Name: pl, dtype: float64
---------------------
max : 3 : 0.07279
min : 10 : -0.016329999999999997
---------------------
monthly_avg : 0.0107


In [4]:
data['lema_gap_pl_analysis_month'] = 8

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

,index,positive,negative,count_diff,positive_sum,negative_sum,sum_diff
3,0.000,0.0,1.0,-1.0,0.00000,-0.00343,-0.00343
2,0.001,2.0,1.0,1.0,0.00551,-0.00441,0.00110
0,0.002,1.0,3.0,-2.0,0.00183,-0.00658,-0.00475
6,0.003,1.0,0.0,1.0,0.00279,0.00000,0.00279
1,0.004,0.0,1.0,-1.0,0.00000,-0.00538,-0.00538
5,0.005,1.0,0.0,1.0,0.00045,0.00000,0.00045
4,0.006,0.0,1.0,-1.0,0.00000,-0.00195,-0.00195


In [5]:
sys.exit()

SystemExit: 

In [ ]:
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl_type'].count().unstack())
print('----------------------------------------------')
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl'].sum().unstack())

In [ ]:
data['lema_gap_pl_analysis_month'] = 3

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

In [ ]:
y = data['df_ohlc']['pl'].sum()
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
send_telegram_message(f'{data["input_year"]}')
send_telegram_message(f'{y}')
send_telegram_message(f'{x}')
send_telegram_message(f'-------------------')

In [ ]:
y = data['df_ohlc'][data['df_ohlc']['month_val'] == 6]
x = y.groupby(['date_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')

In [ ]:
data['df_ohlc'].tail()

In [ ]:
data['df_ohlc']['lema_gap'].describe()

In [ ]:
x = 5
data['df_ohlc'].iloc[-5:,x:x+15]

In [ ]:
data['df_small']['lema_gap'].describe()

In [ ]:
abs(all['negative_sum']) / all['positive_sum']

In [ ]:
df_small['order_side']

In [ ]:
data['df_small'].to_csv('data/temp.csv')

In [123]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
month = 8

# Monthly Chart -------------------
data['plot_start']  = f"{data['input_year']}-{month}-1"
data['plot_stop']   = f"{data['input_year']}-{month+1}-1"
# Monthly Chart -------------------

# December Chart ----------------------------
# data['plot_stop']   = f"{data['input_year']+1}-1-1"
# December Chart ----------------------------

# Year Chart ----------------------------
# data['plot_start']  = f"{data['input_year']}-1-1"
# data['plot_stop']   = f"{data['input_year']+1}-1-1"
# Year Chart ----------------------------

# data['things_to_plot'] = ['tick', 'ema', 'BBands', 'dots']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'indicators']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'cross']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'tip']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'lema_angle']
# data['things_to_plot'] = ['ema', 'sema_angle', 'slema_angle', 'lema_angle', 'dir']
# data['things_to_plot'] = ['ema', 'positions', 'lema_angle','slema_angle','sema_angle', 'rsi']
# data['things_to_plot'] = ['ema','positions', 'lema_angle','lema_diff', 'lema_max_min']
# data['things_to_plot'] = ['candles','sema', 'slema','positions','lema', 'lema_angle', 'lema_max_min', 'close_diff']
# data['things_to_plot'] = ['lema', 'tip', 'candles', 'lema_angle', 'lema_change']
data['things_to_plot'] = ['lema', 'candles', 'lema_angle', 'lema_change']

plot_graph(data)

generate_result_report(data)

In [84]:
x = np.sign(data['df_ohlc']['lema'].diff())
x[0:30]

0     NaN
1     0.0
2    -1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    0.0
18    1.0
19    1.0
20    1.0
21    0.0
22    0.0
23    0.0
24    1.0
25   -1.0
26    1.0
27    1.0
28    1.0
29    1.0
Name: lema, dtype: float64

In [69]:
for i in tqdm(np.arange(1, len(x))):
    if x[i] == 0:
        x[i] = x[i-1]
        
x[0:20]

100%|████████████████████████████████| 74386/74386 [00:00<00:00, 294152.45it/s]


0     NaN
1     NaN
2    -1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
Name: lema, dtype: float64

In [101]:
data['df_ohlc']['lema'].diff()[0:30]

0              NaN
1     0.000000e+00
2    -1.000000e-06
3     3.000000e-06
4     4.000000e-06
5     3.000000e-06
6     4.000000e-06
7     5.000000e-06
8     5.000000e-06
9     6.000000e-06
10    4.000000e-06
11    5.000000e-06
12    5.000000e-06
13    4.000000e-06
14    1.000000e-06
15    3.000000e-06
16    2.000000e-06
17    0.000000e+00
18    1.000000e-06
19    2.000000e-06
20    1.000000e-06
21    0.000000e+00
22    0.000000e+00
23    0.000000e+00
24    1.000000e-06
25   -1.000000e-06
26    1.000000e-06
27    2.000000e-06
28    2.000000e-06
29    1.000000e-06
Name: lema, dtype: float64

In [ ]:
a = np.array(data['df_ohlc']['lema'].diff())

In [93]:
a

array([     nan,  0.0e+00, -1.0e-06, ..., -3.4e-05, -3.3e-05, -3.5e-05])

In [92]:
a[:-1]

array([     nan,  0.0e+00, -1.0e-06, ..., -3.2e-05, -3.4e-05, -3.3e-05])

In [94]:
a[1:]

array([ 0.0e+00, -1.0e-06,  3.0e-06, ..., -3.4e-05, -3.3e-05, -3.5e-05])

In [95]:
a[:-1] * a[1:]

array([       nan, -0.000e+00, -3.000e-12, ...,  1.088e-09,  1.122e-09,
        1.155e-09])

In [96]:
a[:-1] * a[1:] < 0

array([False, False,  True, ..., False, False, False])

In [100]:
idx1

array([    3,    25,    26, ..., 74226, 74228, 74232], dtype=int64)

In [86]:
idx1 = np.where(a[:-1] * a[1:] < 0)[0] +1
a[idx1]

[ 3.e-06 -1.e-06  1.e-06 ... -1.e-06  6.e-06 -1.e-06]


In [110]:
a

array([     nan,  0.0e+00, -1.0e-06, ..., -3.4e-05, -3.3e-05, -3.5e-05])

In [117]:
a1 = np.sign(a)
idx2 = []

for i in idx1:
    if i > 2:
        if a1[i-1] == a1[i-2]:
            if a1[i] == a1[i+1]:
                idx2.append(i)

In [121]:
idx2 = (np.array(idx2))

In [88]:
data['df_ohlc']['lema_change'] = np.nan
data['df_ohlc'].loc[np.where(a[:-1] * a[1:] < 0 )[0] +1, 'lema_change'] = data['df_ohlc']['lema']

In [122]:
data['df_ohlc']['lema_change'] = np.nan
data['df_ohlc'].loc[idx2, 'lema_change'] = data['df_ohlc']['lema']

In [45]:
data['df_ohlc']['lema_change'] = np.nan

data['df_ohlc'].loc[np.sign(data['df_ohlc']['lema'].diff()).ne(0), 'lema_change'] = data['df_ohlc']['lema']

In [46]:
data['df_ohlc'][data['df_ohlc']['lema_change'].notnull()]

,DateTime_frmt,open,high,low,close,candle_size,ask,bid,lema,lema_angle,...,up,down,month_val,date_val,hour_val,long_profit,long_loss,short_profit,short_loss,lema_change
0,2020-01-05 23:45:00,1.116605,1.116605,1.116455,1.116455,0.000150,1.116505,1.116405,1.116154,-0.000340,...,NaN,NaN,1,5,23,NaN,NaN,NaN,NaN,1.116154
2,2020-01-05 23:55:00,1.116155,1.116230,1.116050,1.116060,0.000180,1.116110,1.116010,1.116153,-0.000317,...,NaN,NaN,1,5,23,NaN,NaN,NaN,NaN,1.116153
3,2020-01-06 00:00:00,1.116040,1.116385,1.116015,1.116375,0.000370,1.116425,1.116325,1.116156,-0.000306,...,NaN,NaN,1,6,0,NaN,NaN,NaN,NaN,1.116156
4,2020-01-06 00:05:00,1.116390,1.116725,1.116390,1.116655,0.000335,1.116705,1.116605,1.116160,-0.000295,...,NaN,NaN,1,6,0,NaN,NaN,NaN,NaN,1.116160
5,2020-01-06 00:10:00,1.116650,1.116650,1.116385,1.116400,0.000265,1.116450,1.116350,1.116163,-0.000284,...,NaN,NaN,1,6,0,NaN,NaN,NaN,NaN,1.116163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74382,2020-12-31 21:35:00,1.222015,1.222060,1.221955,1.222050,0.000105,1.222100,1.222000,1.225156,-0.001126,...,NaN,NaN,12,31,21,NaN,NaN,NaN,NaN,1.225156
74383,2020-12-31 21:40:00,1.222010,1.222010,1.221865,1.221900,0.000145,1.221950,1.221850,1.225124,-0.001143,...,NaN,NaN,12,31,21,NaN,NaN,NaN,NaN,1.225124
74384,2020-12-31 21:45:00,1.221895,1.221975,1.221720,1.221730,0.000255,1.221780,1.221680,1.225090,-0.001160,...,NaN,NaN,12,31,21,NaN,NaN,NaN,NaN,1.225090
74385,2020-12-31 21:50:00,1.221740,1.221845,1.221560,1.221805,0.000285,1.221855,1.221755,1.225057,-0.001177,...,NaN,NaN,12,31,21,NaN,NaN,NaN,NaN,1.225057


In [ ]:
data['df_ohlc'].loc[(data['df_ohlc']['month_val'] == 8)
                & (data['df_ohlc']['date_val'] == 13)
                & (data['df_ohlc']['hour_val'] == 15), ['DateTime_frmt', 'lema']]